In [0]:
-- Create a new database named copy_data.
create database copy_data

In [0]:
-- Create a new table for copying training data from S3
CREATE TABLE bank_details_training(
   age numeric,
   jobtype char (25),
   marital char (25),
   education char (25),
   default_col char (25),
   housing char (25),
   loan char (25),
   contact char (25),
   month char (25),
   day_of_week char (25),
   duration numeric,
   campaign numeric,
   pdays numeric,
   previous numeric,
   poutcome char (25),
   emp_var_rate numeric,
   cons_price_idx numeric,     
   cons_conf_idx numeric,     
   euribor3m numeric,
   nr_employed numeric,
   y char(1) ) ;

In [0]:
-- Create a new table for copying inference data from S3
CREATE TABLE bank_details_inference(
   age numeric,
   jobtype char (25),
   marital char (25),
   education char (25),
   default_col char (25),
   housing char (25),
   loan char (25),
   contact char (25),
   month char (25),
   day_of_week char (25),
   duration numeric,
   campaign numeric,
   pdays numeric,
   previous numeric,
   poutcome char (25),
   emp_var_rate numeric,
   cons_price_idx numeric,     
   cons_conf_idx numeric,     
   euribor3m numeric,
   nr_employed numeric,
   y char(1) ) ;

In [0]:
-- Update the placeholder for S3 bucket to point to the snapshot export location. eg: s3://bucket_name/snapshot_export/db_name/training_table_name/
-- Update the placeholder for IAM role to use the role that has access to S3 bucket. Make sure to attach it to the Redshift cluster
-- Run a COPY command to copy training data from S3 to bank_details_training
copy bank_details_training
from '<s3 training bucket name>'
FORMAT AS PARQUET
iam_role 'arn:aws:iam::801214630732:role/iam_role'

In [0]:
-- Update the placeholder for S3 bucket to point to the snapshot export location. eg: s3://bucket_name/snapshot_export/db_name/inference_table_name/
-- Update the placeholder for IAM role to use the role that has access to S3 bucket. Make sure to attach it to the Redshift cluster
-- Run a COPY command to copy training data from S3 to bank_details_inference
copy bank_details_inference
from '<s3 inference bucket name>'
FORMAT AS PARQUET
iam_role 'arn:aws:iam::801214630732:role/iam_role'

In [0]:
-- Validation of training table load
select * from bank_details_training

In [0]:
-- Creating a new training table to add new column for row number. This will be used for including only a subset of records from the original training table for training the ML model
create table bank_details_training_rnum as (select row_number() over (partition by 1) as rnum, t.* from bank_details_training t)

In [0]:
-- Validation of records in the new training table. It should give 4000
select count(*) from bank_details_training_rnum where rnum > 119 order by 1

In [0]:
-- Create ML model. Provide only the bucket name (without any prefix such as s3://)
CREATE MODEL td_subscription_demo
FROM 
(SELECT age
,jobtype
,marital
,education
,default_col
,housing
,loan
,contact
,month
,day_of_week
,duration
,campaign
,pdays
,previous
,poutcome
,emp_var_rate
,cons_price_idx
,cons_conf_idx
,euribor3m
,nr_employed
,y
FROM bank_details_training_rnum
where rnum > 119
)
TARGET y
FUNCTION predict_td_subscription_demo
iam_role 'arn:aws:iam::801214630732:role/iam_role'
SETTINGS (
  S3_BUCKET 'bucket_name'
);

In [0]:
-- Check model status. Wait until the status changes to READY. It takes approx. 1 hour for this sample dataset.
show model td_subscription_demo

In [0]:
-- Check the model accuracy by comparing the output of ML model's prediction and the actual data. 

WITH infer_data
 AS (
    SELECT  y as actual, predict_td_subscription_demo(age,jobtype,marital,education,default_col,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed) AS predicted,
     CASE WHEN actual = predicted THEN 1::INT
         ELSE 0::INT END AS correct
    FROM bank_details_inference
    ),
 aggr_data AS (
     SELECT SUM(correct) as num_correct, COUNT(*) as total FROM infer_data
 )
SELECT cast(num_correct as decimal(10,2)) AS Correct_Entries, cast(total as decimal(10,2)) AS Total_Entries, (Correct_Entries/Total_Entries) AS Accuracy FROM aggr_data;

--Predict how many will subscribe for term deposit vs not subscribe

WITH term_data AS ( SELECT predict_td_subscription_demo( age,jobtype,marital,education,default_col,housing,loan,contact,month,day_of_week,duration,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed) AS predicted
FROM bank_details_inference )
SELECT
CASE WHEN predicted = 'y'  THEN 'Yes-will-do-a-term-deposit'
     WHEN predicted = 'n'  THEN 'No-term-deposit'
     ELSE 'Neither' END as deposit_prediction,
COUNT(1) AS count
from term_data GROUP BY 1;